In [1]:
# pip install dali-dataset

In [2]:
import DALI as dali_code
import pandas as pd
import os
import re

In [3]:
# Relative path to DALI folder
path = "../DALI_v1.0"

# Import DALI dataset
full_path = os.path.abspath(path)
dali_data = dali_code.get_the_DALI_dataset(full_path, skip=[], keep=[])

# Get info
dali_info_path = os.path.join(full_path, 'info/DALI_DATA_INFO.gz')
dali_info = dali_code.get_info(dali_info_path)

print(dali_info)

[['DALI_ID' 'NAME' 'YOUTUBE' 'WORKING']
 ['e186227bb7474fa5a7738c9108f11972' 'Staind-Tangled_Up_In_You'
  'NXG-ayocugI' 'True']
 ['520f583def024997adcab0567fb25a5d' 'Boyzone-Baby_Can_I_Hold_You'
  'ZjSLNZ9MsMI' 'True']
 ...
 ['e64aed27b6d64630985a5bc2ad571383'
  'Green_Day-Good_Riddance_(Time_Of_Your_Life)' '_bTdLi0YUVM' 'True']
 ['a8d53be9df7f41d58b3c7e404c8fc228'
  'Elton_John-Can_You_Feel_The_Love_Tonight?' 'KjgWWjkNbhU' 'True']
 ['a8bd4850b7f14f61bd745a5c82a899f8' 'Blink_182-Man_Overboard'
  'JyNqLOBlkFk' 'True']]


In [4]:
# Make a dataframe for easier viewing
df = pd.DataFrame(dali_info[1:], columns=dali_info[0])
df.sample(10)


,DALI_ID,NAME,YOUTUBE,WORKING
3790,0091064bdc72469ca7096d3a0db74562,Five_For_Fighting-100_Years,fJN9YIOl1xE,True
3462,aacac80a46e34367812444e126d215a3,Emeli_Sandé-Next_To_Me,CtWUU53rt2Y,True
1279,ee64a9a8f6664ceca8b9159a222dbe5b,Olsen_Brothers-Smuk_Som_Et_Stjerneskud,l9Jck3OO5NA,True
823,e9f734ea16ac40da9e26b6f2d5a9b503,Annett_Louisan-Das_Spiel,Iva-mHSSuZs,True
966,93d64c6ee846496ebdcb5e4886756b32,Bruce_Springsteen-My_Hometown,Ic4cMhyVHN0,True
4937,032daa690b61406686d283422a1a2e43,Scooter-Maria,KVjeheaFfsM,True
5222,5209fa31fc5b4f25a22705ea9274951b,Something_Corporate-Down,3vunvBYQjHA,True
4049,8865b9a3a065459e9e045689699f66d6,Marco_Mengoni-Ti_Ho_Voluto_Bene_Veramente,ARqpqyA49y0,True
1659,b871069da8d546feb83de094b6d8a685,System_Of_A_Down-Cigaro,5XL-Rr5a-Nk,True
3525,817979d857ca4a629b1ba4415c723997,"Peter,_Paul_&_Mary-500_Miles",ADN1lLEp3H0,True


In [5]:
df[df['DALI_ID']=='50e11b8e44024823ae9fe4bc38d18ea5']

,DALI_ID,NAME,YOUTUBE,WORKING
2299,50e11b8e44024823ae9fe4bc38d18ea5,KoRn-Here_To_Stay,Ft-7at-gNTU,True


In [6]:
entry = dali_data['50e11b8e44024823ae9fe4bc38d18ea5']
entry.info

{'dataset_version': 1.0,
 'ground-truth': False,
 'artist': 'KoRn',
 'title': 'Here To Stay',
 'scores': {'NCC': 0.9200032203657332, 'manual': 0.0},
 'audio': {'url': 'Ft-7at-gNTU', 'path': 'None', 'working': True},
 'id': '50e11b8e44024823ae9fe4bc38d18ea5',
 'metadata': {'album': 'Untouchables',
  'release_date': '2002',
  'cover': 'https://e-cdns-images.dzcdn.net/images/cover/6999e41d627f501bf0b588860405e2c7/1000x1000-000000-80-0-0.jpg',
  'genres': ['Pop', 'Metal'],
  'language': 'english'}}

In [8]:
# Use re to remove punctuation and force lowercase
def remove_punctuation(s):
    s = re.sub(r'[^a-zA-Z0-9\s]', '', s)
    return s.lower()

to_add = []

## Turn lyrics into dataframe
for d in entry.annotations['annot']['words']:
    word = remove_punctuation(d['text'])
    start = d['time'][0]
    end = d['time'][1]
    to_add.append([word, start, end])

df_entry = pd.DataFrame(to_add, columns=['word', 'start', 'end'])
print(df_entry)

# save lyrics to df
file_name = f"../DALI-audio/{entry.info['id']}/lyrics.csv"
os.makedirs(f"../DALI-audio/{entry.info['id']}", exist_ok=True)

df_entry.to_csv(os.path.abspath(file_name), index=False)

        word    start      end
0       this   51.514   51.814
1       time   51.814   52.414
2     taking   52.414   53.014
3         it   53.014   53.314
4       away   53.314   53.914
..       ...      ...      ...
323  anymore  247.114  248.014
324     give  251.614  251.914
325  anymore  251.914  252.814
326     give  256.414  256.714
327  anymore  256.714  257.614

[328 rows x 3 columns]


In [ ]:
import yt_dlp

def download_youtube_audio(url, filename, output_path='.', file_format='mp3'):
    """
    Downloads audio from a YouTube URL.

    Args:
        url (str): The YouTube video URL.
        output_path (str): The directory to save the audio file.
        file_format (str): The desired audio format (e.g., 'mp3', 'wav', 'm4a', 'opus').
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    ## ffmpeg location varies by operating system...
    if os.name == 'nt':
        ffmpeg_loc = "C:\\ffmpeg\\bin\\"

    elif os.name == 'posix':
        ffmpeg_loc = "/opt/anaconda3/bin/ffmpeg"
    
    ydl_opts = {
        'format': 'bestaudio/best',  # Download the best available audio format
        'outtmpl': os.path.join(output_path, f"{filename}"), # Output file template
        'postprocessors': [{
            'key': 'FFmpegExtractAudio', # Use FFmpeg to extract audio
            'preferredcodec': file_format, # Preferred audio codec
            'preferredquality': '0',     # Best quality
        }],
        'ffmpeg_location': ffmpeg_loc,
        'quiet': True, # Turns off text info
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            #title = info_dict.get('title', 'unknown_title')
            #print(f"Successfully downloaded: {filename}.{file_format}")
            return os.path.join(output_path, f"{filename}.{file_format}") # Return the downloaded file path
        
    except yt_dlp.DownloadError as e:
        print(f"Error downloading audio: {e}")
        return None
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


In [ ]:
# Get audio from youtube and save
youtube_url = entry.info['audio']['url']
output_dir = os.path.abspath(f"../DALI-audio/{entry.info['id']}")

downloaded_file = download_youtube_audio(url=youtube_url, 
                                         filename='audio',
                                         output_path=output_dir,
                                         file_format='mp3')

if downloaded_file:
    print(f"Audio saved to: {downloaded_file}")
    
else:
    print("Audio download failed.")

Audio saved to: /Users/duncanclark/Documents/local/DALI-audio/50e11b8e44024823ae9fe4bc38d18ea5/audio.mp3


In [ ]:
import pickle

def save_dictionary(dictionary_to_save, filename):
    with open(filename, 'wb') as f: # 'wb' mode for writing in binary
        pickle.dump(dictionary_to_save, f)
    return

# and save dict
save_dictionary(entry.info, os.path.abspath(f"../DALI-audio/{entry.info['id']}/metadata.pkl"))

Apply to the whole dataset (it will take awhile)

In [ ]:
## Now do it all in one go
for entry_id in df['DALI_ID']:
    
    entry = dali_data[entry_id]
    output_dir = os.path.abspath(f"../DALI-audio/{entry.info['id']}")
    
    fname = remove_punctuation(entry.info['title']).replace(" ", "-") ### Check this

    ## Create directory
    os.makedirs(output_dir, exist_ok=True)

    ## Get audio
    youtube_url = entry.info['audio']['url']

    downloaded_file = download_youtube_audio(url=youtube_url, 
                                            filename=fname,
                                            output_path=output_dir,
                                            file_format='wav')

    # If the youtube download fails remove the directory and continue
    if not downloaded_file:
        os.rmdir(output_dir)
        continue
    
    ## Turn lyrics into dataframe
    to_add = []
    for d in entry.annotations['annot']['words']:
        word = remove_punctuation(d['text'])
        
        start = d['time'][0]
        end = d['time'][1]

        to_add.append([word, start, end])

    df_entry = pd.DataFrame(to_add, columns=['word', 'start', 'end'])

    # Save lyrics to df
    df_entry.to_csv(os.path.abspath(f"../DALI-audio/{entry.info['id']}/lyrics.csv"), index=False)
    
    ## Save metadata
    save_dictionary(entry.info, os.path.abspath(f"../DALI-audio/{entry.info['id']}/metadata.pkl"))

[youtube] Extracting URL: NXG-ayocugI
[youtube] NXG-ayocugI: Downloading webpage
[youtube] NXG-ayocugI: Downloading tv client config
[youtube] NXG-ayocugI: Downloading tv player API JSON
[youtube] NXG-ayocugI: Downloading ios player API JSON
[youtube] NXG-ayocugI: Downloading m3u8 information
[info] NXG-ayocugI: Downloading 1 format(s): 251
[download] Destination: ..\DALI-audio\e186227bb7474fa5a7738c9108f11972\tangled-up-in-you
[download] 100% of    4.77MiB in 00:00:00 at 19.04MiB/s    
[ExtractAudio] Destination: ..\DALI-audio\e186227bb7474fa5a7738c9108f11972\tangled-up-in-you.wav
Deleting original file ..\DALI-audio\e186227bb7474fa5a7738c9108f11972\tangled-up-in-you (pass -k to keep)
[youtube] Extracting URL: ZjSLNZ9MsMI
[youtube] ZjSLNZ9MsMI: Downloading webpage
[youtube] ZjSLNZ9MsMI: Downloading tv client config
[youtube] ZjSLNZ9MsMI: Downloading tv player API JSON
[youtube] ZjSLNZ9MsMI: Downloading ios player API JSON
[youtube] ZjSLNZ9MsMI: Downloading m3u8 information
[info] ZjSL

ERROR: [youtube] aJCOAZMn9pU: Video unavailable


Error downloading audio: ERROR: [youtube] aJCOAZMn9pU: Video unavailable
Doesn't exit, skipping
[youtube] Extracting URL: lEUy2cGJwG0
[youtube] lEUy2cGJwG0: Downloading webpage
[youtube] lEUy2cGJwG0: Downloading tv client config
[youtube] lEUy2cGJwG0: Downloading tv player API JSON
[youtube] lEUy2cGJwG0: Downloading ios player API JSON


ERROR: [youtube] lEUy2cGJwG0: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Error downloading audio: ERROR: [youtube] lEUy2cGJwG0: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
Doesn't exit, skipping
[youtube] Extracting URL: PbVYg0EVpek
[youtube] PbVYg0EVpek: Downloading webpage
[youtube] PbVYg0EVpek: Downloading tv client config
[youtube] PbVYg0EVpek: Downloading tv player API JSON
[youtube] PbVYg0EVpek: Downloading ios player API JSON


...

ERROR: [youtube] CIYS9EQWkXg: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Error downloading audio: ERROR: [youtube] CIYS9EQWkXg: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
Doesn't exit, skipping
[youtube] Extracting URL: ACp-kVgV0Iw
[youtube] ACp-kVgV0Iw: Downloading webpage
[youtube] ACp-kVgV0Iw: Downloading tv client config
[youtube] ACp-kVgV0Iw: Downloading tv player API JSON
[youtube] ACp-kVgV0Iw: Downloading ios player API JSON
[youtube] ACp-kVgV0Iw: Downloading m3u8 information
[info] ACp-kVgV0Iw: Downloading 1 format(s): 251
[download] Destination: ..\DALI-audio\81275468fa124185a222bf037764e925\ring-of-steel
[download] 100% of    6.35MiB in 00:00:00 at 18.29MiB/s  
[ExtractAudio] Destination: ..\DALI-audio\81275468fa124185a222bf037764e925\ring-of-steel.wav
Deleting original file ..\DALI-audio\81275468fa124185a222bf037764e925\ring-of-steel (pass -k to keep)
[youtube] Extracting URL: bzYHbPkrMzw
[youtube] bzYHbPkrMzw: Downloading webpage
[youtube] bzYHbPkrMzw: Download

ERROR: [youtube] iIWHIvaK7DA: Video unavailable. This video is no longer available due to a copyright claim by Ex Noctem Nacimur Music


Error downloading audio: ERROR: [youtube] iIWHIvaK7DA: Video unavailable. This video is no longer available due to a copyright claim by Ex Noctem Nacimur Music
Doesn't exit, skipping
[youtube] Extracting URL: xntxoEFsqfU
[youtube] xntxoEFsqfU: Downloading webpage
[youtube] xntxoEFsqfU: Downloading tv client config
[youtube] xntxoEFsqfU: Downloading tv player API JSON
[youtube] xntxoEFsqfU: Downloading ios player API JSON
[youtube] xntxoEFsqfU: Downloading m3u8 information
[info] xntxoEFsqfU: Downloading 1 format(s): 251
[download] Destination: ..\DALI-audio\3852f692f72e4c43bbcdb6fe021ea087\ill-be-waiting
[download] 100% of    4.35MiB in 00:00:00 at 18.92MiB/s  
[ExtractAudio] Destination: ..\DALI-audio\3852f692f72e4c43bbcdb6fe021ea087\ill-be-waiting.wav
Deleting original file ..\DALI-audio\3852f692f72e4c43bbcdb6fe021ea087\ill-be-waiting (pass -k to keep)
[youtube] Extracting URL: bhUTNN7AB6k
[youtube] bhUTNN7AB6k: Downloading webpage
[youtube] bhUTNN7AB6k: Downloading tv client config


[youtube] bhUTNN7AB6k: Downloading m3u8 information
[info] bhUTNN7AB6k: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 41
[download] Destination: ..\DALI-audio\f1d78243e60346cca09a17468220e6c8\angus-mcfife
[download] 100% of    3.28MiB in 00:00:01 at 2.10MiB/s                   
[ExtractAudio] Destination: ..\DALI-audio\f1d78243e60346cca09a17468220e6c8\angus-mcfife.wav
Deleting original file ..\DALI-audio\f1d78243e60346cca09a17468220e6c8\angus-mcfife (pass -k to keep)
[youtube] Extracting URL: OQHtHowsRC4
[youtube] OQHtHowsRC4: Downloading webpage
[youtube] OQHtHowsRC4: Downloading tv client config
[youtube] OQHtHowsRC4: Downloading tv player API JSON
[youtube] OQHtHowsRC4: Downloading ios player API JSON
[youtube] OQHtHowsRC4: Downloading m3u8 information
[info] OQHtHowsRC4: Downloading 1 format(s): 251
[download] Destination: ..\DALI-audio\1d87a1b3629f435c84ecd243e7fa6cb0\wishmaster-live
[download] 100% of    3.92MiB in 00:00:00 at 13.